In [7]:
import pandas as pd
import gmaps
import gmaps.datasets
import os

project_id = "the-d-team-164512"
gmaps.configure(api_key=os.environ['GMAP_API_THE_D_TEAM'])

query = """\
SELECT
t_lipitor.year
, t_lipitor.patient_id
, p.postcode
, pc.lat
, pc.lon
FROM(
SELECT
  t.Patient_ID AS patient_id
  , YEAR(DATE(t.Dispense_Week)) AS year
FROM [the-d-team-164512:dthon.transactions] t
JOIN [the-d-team-164512:dthon.drug_lookup] d
ON t.Drug_ID = d.MasterProductID
WHERE
  d.BrandName = 'LIPITOR'
GROUP BY
  patient_id
  , year
) t_lipitor
JOIN [the-d-team-164512:dthon.patients_lookup] p
ON t_lipitor.Patient_ID = p.Patient_ID
LEFT JOIN (
  SELECT postcode, FIRST(lat) AS lat, FIRST(lon) AS lon
  FROM [the-d-team-164512:dthon.postcodes]
  GROUP BY postcode
  ) pc
ON p.postcode = pc.postcode
ORDER BY year ASC
"""

df = pd.read_gbq(query, project_id)
df.head(3)

Waiting for job to complete...


,t_lipitor_year,t_lipitor_patient_id,p_postcode,pc_lat,pc_lon
0,2016,174367,2168,-33.917587,150.899095
1,2016,114079,4350,-27.600764,151.914462
2,2016,249835,3178,-37.928005,145.235811


In [58]:

locations = df[['pc_lat', 'pc_lon']]
locations = locations.dropna() # presumably due to not found postcode

m = gmaps.Map()
heatmap_layer = gmaps.heatmap_layer(locations)
heatmap_layer.max_intensity = 100 # prevent dissaption on zoom
heatmap_layer.point_radius = 5
m.add_layer(heatmap_layer)
m

In [59]:
m = gmaps.Map()
heatmap_layer_2011 = gmaps.heatmap_layer(df[df['t_lipitor_year'] == 2011][['pc_lat', 'pc_lon']].dropna())
heatmap_layer_2011.max_intensity = 100 # prevent dissaption on zoom
heatmap_layer_2011.point_radius = 10
m.add_layer(heatmap_layer_2011)

# TODO is there a way to have mutliple layers and turn them on and off??? and have each year in layers? or animate
# heatmap_layer_2016 = gmaps.heatmap_layer(df[df['t_lipitor_year'] == 2016][['pc_lat', 'pc_lon']].dropna())
# heatmap_layer_2016.max_intensity = 100 # prevent dissaption on zoom
# heatmap_layer_2016.point_radius = 5
# m.add_layer(heatmap_layer_2016)
m

In [60]:
m = gmaps.Map()
heatmap_layer_2016 = gmaps.heatmap_layer(df[df['t_lipitor_year'] == 2015][['pc_lat', 'pc_lon']].dropna())
heatmap_layer_2016.max_intensity = 100 # prevent dissaption on zoom
heatmap_layer_2016.point_radius = 10
m.add_layer(heatmap_layer_2016)
m